In [3]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [4]:
# !cd '/content/drive/MyDrive'
import os
# # os.chdir('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis')
# # os.chdir('/content/drive')
# import torch '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF without DA/12k Drive End Bearing Fault Data'
datafiles = os.listdir('/content/drive/MyDrive/')
print(datafiles)

['Colab Notebooks', 'Experiment', 'Colab-Notebooks']


In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
!pip install pyts --target=/content/drive/MyDrive/Colab-Notebooks/python_packages

In [1]:
!nvidia-smi

Tue Oct 18 22:35:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# 测试两个type和两个severity，GAF/MTF，CNN

# focus on file '12k Drive End Bearing Fault Data'

import os, sys
import scipy.io as io
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pylab
from PIL import Image
from torch.autograd import Variable
from tqdm import tqdm
import logging
import time

plt.rc('font',family='Times New Roman')

# GPU
device='cuda:0'

# Test
# choice = 7
for choice in range(1,6+1):

  print("choice",choice)

  # Change the path here to test different data sources and data augmentation methods.
  # Single
  path1 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Drive End Bearing Fault Data'   
  path2 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Fan End Bearing Fault Data'
  path3 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/48k Drive End Bearing Fault Data'
  # MTL
  path4 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Drive End Bearing Fault Data'   
  path5 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/12k Fan End Bearing Fault Data'
  path6 = '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Experiment 1/GAF-MTF DA/48k Drive End Bearing Fault Data'
  paths = [path1, path2, path3, path4, path5, path6]
  path = paths[choice-1]

  # Edit the corresponding testtype.
  testtype1 = 'Ex1_12kDE_Single'
  testtype2 = 'Ex1_12FE_Single'
  testtype3 = 'Ex1_48kDE_Single'
  testtype4 = 'Ex1_12kDE_MTL'
  testtype5 = 'Ex1_12FE_MTL'
  testtype6 = 'Ex1_48kDE_MTL'
  testtypes= [testtype1, testtype2, testtype3, testtype4, testtype5, testtype6]
  testtype = testtypes[choice-1]

  
  def gen_data():
      f={'Ball':0,'Inner':1,'Outer':2,'7':0,'14':1,'21':2,'28':3}

      datafiles = os.listdir(path)
      lentrain = int(len(datafiles) * 0.7)
      lentest = len(datafiles) - lentrain
      X, Y1, Y2=[], [], []
      for file in tqdm(datafiles):
          dataGAF = io.loadmat(os.path.join(path, file))['GAF']
          dataMTF = io.loadmat(os.path.join(path, file))['MTF']
          data = [dataGAF, dataMTF]
          y1, y2, _ = file.split('_')
          Y1.append(f[y1])
          Y2.append(f[y2])
          X.append(data)

      X = torch.tensor(np.array(X).reshape((len(X),2, 256, 256)))
      Y1 = torch.tensor(np.array(Y1).reshape((len(X), 1))).to(torch.long)
      Y2 = torch.tensor(np.array(Y2).reshape((len(X), 1))).to(torch.long)

      n=len(X)
      idx = torch.randperm(n)
      X_train = X[idx[:lentrain],]
      X_test = X[idx[lentrain:],]
      Y1_train = Y1[idx[:lentrain],]
      Y1_test = Y1[idx[lentrain:],]
      Y2_train = Y2[idx[:lentrain],]
      Y2_test = Y2[idx[lentrain:],]

      return X_train, Y1_train, Y2_train, X_test, Y1_test, Y2_test

  

  class Net(nn.Module):
      def __init__(self):
          super(Net, self).__init__()

          # channel = 1 if choice<=12 else 2
          self.conv = nn.Conv2d(2, 8, kernel_size=15, padding=7) 
          self.conv1 = nn.Conv2d(8, 16, kernel_size=5, padding=2) 
          self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=2) 
          self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
          self.conv4 = nn.Conv2d(16, 32, kernel_size=3, padding=1) 

          self.mp = nn.MaxPool2d(2)
          self.relu = nn.ReLU()
          self.tanh = nn.Tanh()
          self.fc1 = nn.Linear(32*32*32,1000)
          self.fc2 = nn.Linear(32*32*32,1000)
          self.fc3 = nn.Linear(1000,100)
          self.fc4 = nn.Linear(1000,100)
          self.fc5 = nn.Linear(100,3)
          self.fc6 = nn.Linear(100,3)
          self.sigmod=nn.Sigmoid()
          self.logsoftmax = nn.LogSoftmax()
          self.softmax = nn.Softmax(dim=-1)

      def forward(self, x):
          out = self.mp(self.conv(x))

          out1 = self.mp(self.conv1(out))
          out1 = self.mp(self.conv3(out1))
          out1 = out1.view(-1)
          out1 = self.sigmod(self.fc1(out1))
          out1 = self.sigmod(self.fc3(out1))
          out1 = self.sigmod(self.fc5(out1))

          out2 = self.mp(self.conv2(out))
          out2 = self.mp(self.conv4(out2))
          out2 = out2.view(-1)
          out2 = self.sigmod(self.fc2(out2))
          out2 = self.sigmod(self.fc4(out2))
          out2 = self.sigmod(self.fc6(out2))

          return out1, out2

  X_train, Y1_train, Y2_train, X_test, Y1_test, Y2_test = gen_data()

  X_train=torch.tensor(X_train).to(device)
  Y1_train=torch.tensor(Y1_train).to(device)
  Y2_train=torch.tensor(Y2_train).to(device)
  X_test=torch.tensor(X_test).to(device)
  Y1_test=torch.tensor(Y1_test).to(device)
  Y2_test=torch.tensor(Y2_test).to(device)



  model = Net()
  model=model.to(device)
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

  if choice>=4 and choice<=6:
  # MTL 

    logs = str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tstart training!'

    Max_epoch = 40
    acclist = []
    for epoch in range(Max_epoch):
        print('*' * 25, 'epoch {}'.format(epoch + 1), '*' * 25)
        running_loss1 = 0.0
        running_loss2 = 0.0
        running_acc1_train = 0.0
        running_acc2_train = 0.0
        n = len(X_test)
        w1, w2 = 0.5, 0.5
        running_acc1_test = 0.0
        running_acc2_test = 0.0

        for t,x in tqdm(enumerate(X_train)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_train[t]).to(torch.long), Variable(Y2_train[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            model.train()
            optimizer.zero_grad()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            loss = w1*loss1 + w2*loss2
            running_loss1 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_train = (pred1 == b_y1).sum()
            num_correct2_train = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_train += num_correct1_train.item()
            running_acc2_train += num_correct2_train.item()
            # print(running_acc1, running_acc2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        for t,x in tqdm(enumerate(X_test)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_test[t]).to(torch.long), Variable(Y2_test[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            running_loss2 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_test = (pred1 == b_y1).sum()
            num_correct2_test = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_test += num_correct1_test.item()
            running_acc2_test += num_correct2_test.item()
            # print(running_acc1, running_acc2)

        print('Finish {} epoch, Loss_train: {:.6f}, Loss_test: {:.6f}, Acc1_train: {:.6f}, Acc2_train: {:.6f}, Acc1_test: {:.6f}, Acc2_test: {:.6f}'.format(
            epoch + 1, running_loss1 / len(X_train), running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)))
        logs += '\n'+str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tEpoch:[{}/{}]\t loss_train={:.5f}\t loss_test={:.5f}\t Acc1_train={:.5f}\t Acc2_train={:.5f}\t Acc1_test={:.5f}\t Acc2_test={:.5f}'.\
                    format(epoch+1 , Max_epoch, running_loss1 / len(X_train),running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)  )
        acclist.append([(running_acc1_test / len(X_test)+ running_acc2_test / len(X_test))/2,epoch])
        # torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))

    logs += '\n' + str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())) + '\tfinish training!'
    acclist.sort(reverse=True)
    best_epoch=acclist[0][1]
    # for epoch in range(Max_epoch):
    #   if epoch==best_epoch:
    #     model = torch.load('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    #     torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)
    #   os.remove('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    # testtype = 'Ex1_12kDE_withoutDA'

    logs += '\nBest epoch:'+str(best_epoch+1)
    with open('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Logs/'+testtype+'.log','w+') as f:
        f.write(logs)
    torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)
  else: 
    # Single task
    # task 1
    logs = str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tstart training!'

    Max_epoch = 40
    acclist = []
    for epoch in range(Max_epoch):
        print('*' * 25, 'epoch {}'.format(epoch + 1), '*' * 25)
        running_loss1 = 0.0
        running_loss2 = 0.0
        running_acc1_train = 0.0
        running_acc2_train = 0.0
        n = len(X_test)
        w1, w2 = 1, 0
        running_acc1_test = 0.0
        running_acc2_test = 0.0

        for t,x in tqdm(enumerate(X_train)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_train[t]).to(torch.long), Variable(Y2_train[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            model.train()
            optimizer.zero_grad()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            loss = w1*loss1 + w2*loss2
            running_loss1 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_train = (pred1 == b_y1).sum()
            num_correct2_train = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_train += num_correct1_train.item()
            running_acc2_train += num_correct2_train.item()
            # print(running_acc1, running_acc2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        for t,x in tqdm(enumerate(X_test)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_test[t]).to(torch.long), Variable(Y2_test[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            running_loss2 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_test = (pred1 == b_y1).sum()
            num_correct2_test = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_test += num_correct1_test.item()
            running_acc2_test += num_correct2_test.item()
            # print(running_acc1, running_acc2)

        print('Finish {} epoch, Loss_train: {:.6f}, Loss_test: {:.6f}, Acc1_train: {:.6f}, Acc2_train: {:.6f}, Acc1_test: {:.6f}, Acc2_test: {:.6f}'.format(
            epoch + 1, running_loss1 / len(X_train), running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)))
        logs += '\n'+str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tEpoch:[{}/{}]\t loss_train={:.5f}\t loss_test={:.5f}\t Acc1_train={:.5f}\t Acc2_train={:.5f}\t Acc1_test={:.5f}\t Acc2_test={:.5f}'.\
                    format(epoch+1 , Max_epoch, running_loss1 / len(X_train),running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)  )
        acclist.append([(running_acc1_test / len(X_test)+ running_acc2_test / len(X_test))/2,epoch])
        # torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))

    logs += '\n' + str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())) + '\tfinish training!'
    acclist.sort(reverse=True)
    best_epoch=acclist[0][1]
    # for epoch in range(Max_epoch):
    #   if epoch==best_epoch:
    #     model = torch.load('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    #     torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)
    #   os.remove('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    # testtype = 'Ex1_12kDE_withoutDA'

    logs += '\nBest epoch:'+str(best_epoch+1)
    with open('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Logs/'+testtype+'_1.log','w+') as f:
        f.write(logs)
    torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+'_1')

    #task 2
    logs = str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tstart training!'

    Max_epoch = 40
    acclist = []
    for epoch in range(Max_epoch):
        print('*' * 25, 'epoch {}'.format(epoch + 1), '*' * 25)
        running_loss1 = 0.0
        running_loss2 = 0.0
        running_acc1_train = 0.0
        running_acc2_train = 0.0
        n = len(X_test)
        w1, w2 = 0, 1
        running_acc1_test = 0.0
        running_acc2_test = 0.0

        for t,x in tqdm(enumerate(X_train)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_train[t]).to(torch.long), Variable(Y2_train[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            model.train()
            optimizer.zero_grad()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            loss = w1*loss1 + w2*loss2
            running_loss1 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_train = (pred1 == b_y1).sum()
            num_correct2_train = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_train += num_correct1_train.item()
            running_acc2_train += num_correct2_train.item()
            # print(running_acc1, running_acc2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        for t,x in tqdm(enumerate(X_test)):
            data, target1, target2 = Variable(x.reshape((1,) + x.shape)), Variable(Y1_test[t]).to(torch.long), Variable(Y2_test[t]).to(torch.long)
            # b_x,b_y,b_y1 = data.cuda(),target1.cuda(),target2.cuda()
            b_x, b_y1, b_y2 = data, target1, target2
            b_x = b_x.float()
            out,out1 = model(b_x)
            out = out.reshape((1,len(out)))
            out1 = out1.reshape((1, len(out1)))
            loss1 = loss_fn(out, b_y1)
            loss2 = loss_fn(out1,b_y2)
            running_loss2 += loss1.item() * b_y1.size(0)+loss2.item() * b_y2.size(0)
            _, pred1 = torch.max(out, 1)
            _, pred2 = torch.max(out1, 1)
            num_correct1_test = (pred1 == b_y1).sum()
            num_correct2_test = (pred2 == b_y2).sum()
            # print(num_correct1, num_correct2)
            accuracy = (pred2 == b_y2).float().mean()
            running_acc1_test += num_correct1_test.item()
            running_acc2_test += num_correct2_test.item()
            # print(running_acc1, running_acc2)

        print('Finish {} epoch, Loss_train: {:.6f}, Loss_test: {:.6f}, Acc1_train: {:.6f}, Acc2_train: {:.6f}, Acc1_test: {:.6f}, Acc2_test: {:.6f}'.format(
            epoch + 1, running_loss1 / len(X_train), running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)))
        logs += '\n'+str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()))+'\tEpoch:[{}/{}]\t loss_train={:.5f}\t loss_test={:.5f}\t Acc1_train={:.5f}\t Acc2_train={:.5f}\t Acc1_test={:.5f}\t Acc2_test={:.5f}'.\
                    format(epoch+1 , Max_epoch, running_loss1 / len(X_train),running_loss2 / len(X_test), running_acc1_train / len(X_train), running_acc2_train / len(X_train), running_acc1_test / len(X_test), running_acc2_test / len(X_test)  )
        acclist.append([(running_acc1_test / len(X_test)+ running_acc2_test / len(X_test))/2,epoch])
        # torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))

    logs += '\n' + str(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())) + '\tfinish training!'
    acclist.sort(reverse=True)
    best_epoch=acclist[0][1]
    # for epoch in range(Max_epoch):
    #   if epoch==best_epoch:
    #     model = torch.load('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    #     torch.save(model,'/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype)
    #   os.remove('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+str(epoch))
    # testtype = 'Ex1_12kDE_withoutDA'

    logs += '\nBest epoch:'+str(best_epoch+1)
    with open('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Logs/'+testtype+'_2.log','w+') as f:
        f.write(logs)
    torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis/Models/'+testtype+'_2')


choice 1


100%|██████████| 2700/2700 [00:23<00:00, 113.78it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:133: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:135: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().det

************************* epoch 1 *************************


1889it [01:11, 26.30it/s]
811it [00:02, 391.26it/s]


Finish 1 epoch, Loss_train: 1.971647, Loss_test: 1.908500, Acc1_train: 0.679725, Acc2_train: 0.328745, Acc1_test: 0.733662, Acc2_test: 0.344020
************************* epoch 2 *************************


...

Finish 38 epoch, Loss_train: 1.110114, Loss_test: 1.367131, Acc1_train: 1.000000, Acc2_train: 0.993647, Acc1_test: 0.869297, Acc2_test: 0.855734
************************* epoch 39 *************************


1889it [01:03, 29.60it/s]
811it [00:02, 391.62it/s]


Finish 39 epoch, Loss_train: 1.108209, Loss_test: 1.351891, Acc1_train: 1.000000, Acc2_train: 0.994706, Acc1_test: 0.868064, Acc2_test: 0.866831
************************* epoch 40 *************************


1889it [01:03, 29.60it/s]
811it [00:02, 390.17it/s]


Finish 40 epoch, Loss_train: 1.114212, Loss_test: 1.340556, Acc1_train: 0.996824, Acc2_train: 0.993647, Acc1_test: 0.880395, Acc2_test: 0.877928


In [ ]:
# 读取谷歌云盘的数据请求
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=False)

Drive not mounted, so nothing to flush and unmount.


ValueError: ignored

In [ ]:
import os, sys
path = os.getcwd()
datafiles = os.listdir('/content/drive/MyDrive/Colab Notebooks/Bearing Fault Diagnosis')
print(path)


/content
